#### Test mynds packages in the sandbox

In [1]:
from pathlib import Path

import mynd.backend.metashape as backend

from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


def main() -> None:
    """Main entrypoint."""

    # NOTE: Write your code here
    backend.load_project()


# INVOKE MAIN
main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


TypeError: load_project() missing 1 required positional argument: 'path'